In [1]:
import sys
from pathlib import Path
import pandas as pd

PROJECT_ROOT = Path.cwd().resolve().parents[0]
sys.path.insert(0, str(PROJECT_ROOT))

In [6]:
from src.cohort import load_aki_cohort
from src.utils import (
    add_icu_los_days,
    add_dialysis_flag,
    add_early_dopamine_flag,
    add_sofa_score,
    add_sapsii_score,
    add_vasopressor_flags,
    add_mechanical_ventilation_flag,
    add_early_late_dialysis_flags,
    extract_dialysis_timing,              # falls in src.utils vorhanden
    add_dialysis_near_icu_discharge_flag,
    recode_ethnicity,
    add_rrt_persistence_near_discharge,
)

# 1) Load cohort
df_aki = load_aki_cohort()

# 2) Ensure time columns are datetime (super wichtig!)
for c in ["intime", "outtime", "admittime", "dischtime", "deathtime"]:
    if c in df_aki.columns:
        df_aki[c] = df_aki[c].astype("datetime64[ns]")

# 3) Add baseline / outcomes
df_aki = add_icu_los_days(df_aki)

# 4) Dialysis "ever" flag (pragmatic; ICD + MV)
df_aki = add_dialysis_flag(df_aki)

# 5) SOFA + interventions
df_aki = add_sofa_score(df_aki)
df_aki = add_sapsii_score(df_aki)
df_aki = add_mechanical_ventilation_flag(df_aki)

# 6) Vasopressors / dopamine early flags
df_aki = add_vasopressor_flags(df_aki, window_hours=24)
df_aki = add_early_dopamine_flag(df_aki, window_hours=24)

# 7) Ethnicity grouping
df_aki = recode_ethnicity(df_aki)   # sollte eine neue Spalte wie ethnicity_grp erzeugen

# 8) Timing-aware dialysis (nur MV Events)
df_aki = add_early_late_dialysis_flags(
    df_aki,
    window_hours=24,
    include_inputevents=True,
    allow_negative_hours=False
)

# 9) Optional: Dialysis start/end/duration (falls du die Funktion hast)
# df_aki = extract_dialysis_timing(df_aki)

# 10) Dialysis near ICU discharge (last 6h)
df_aki = add_dialysis_near_icu_discharge_flag(
    df_aki,
    hours_before_discharge=6,
    include_inputevents=True
)
df_aki = extract_dialysis_timing(df_aki)
df_aki = add_rrt_persistence_near_discharge(
    df_aki,
    hours_before_discharge=6
) 

# 11) Quick sanity checks
print("Rows:", len(df_aki))
print("\n" + "="*40)
print("SAPS II PLAUSIBILITÄTS-CHECK")
print("="*40)

# Voraussetzung: 'hospital_mortality' muss vorhanden sein. 
# Falls nicht, muss es aus 'admissions' geholt oder berechnet werden.
if 'hospital_mortality' not in df_aki.columns:
    print("ACHTUNG: 'hospital_expire_flag' fehlt für den Vergleich!")
else:
    # A) Vergleich der Mittelwerte (Score bei Überlebenden vs. Verstorbenen)
    print("\n--- Durchschnittlicher SAPS II Score ---")
    print(df_aki.groupby('hospital_mortality')['sapsii'].agg(['mean', 'median', 'std', 'count']))
    # Erwartung: Verstorbene (Flag=1) sollten deutlich höheren Score haben als Überlebende (Flag=0).

    # B) SMR (Standardized Mortality Ratio)
    # SMR = Beobachtete Mortalität / Vorhergesagte Mortalität

    obs_mortality = df_aki['hospital_mortality'].mean()
    pred_mortality = df_aki['sapsii_prob'].mean()
    smr = obs_mortality / pred_mortality

    print(f"\n--- Global Metrics ---")
    print(f"Beobachtete Mortalität:  {obs_mortality:.2%}")
    print(f"Vorhergesagte Mortalität: {pred_mortality:.2%}")
    print(f"SMR (Observed/Predicted): {smr:.3f}")
    print("-> SMR ≈ 1.0: Score passt perfekt.")
    print("-> SMR < 1.0: Weniger Patienten gestorben als vorhergesagt (gute Performance/Score überschätzt Risiko).")
    print("-> SMR > 1.0: Mehr Patienten gestorben als vorhergesagt (Score unterschätzt Risiko).")

    # C) Kalibrierung (Calibration Table)
    # Wir gruppieren die Patienten nach ihrer vorhergesagten Wahrscheinlichkeit (in 10%-Schritten oder Dezilen)
    # und schauen, wie viele in dieser Gruppe tatsächlich gestorben sind.
    try:
        # Erstelle Bins basierend auf der Wahrscheinlichkeit (Deciles)
        df_aki['prob_bin'] = pd.qcut(df_aki['sapsii_prob'], q=5, duplicates='drop') # 5 Bins reichen oft für den Überblick
        
        calibration = df_aki.groupby('prob_bin', observed=False)[['sapsii_prob', 'hospital_expire_flag']].mean()
        calibration.columns = ['Ø Predicted Prob', 'Ø Observed Mortality']
        calibration['Difference'] = calibration['Ø Observed Mortality'] - calibration['Ø Predicted Prob']
        
        print("\n--- Kalibrierung (Predicted vs. Actual in 5 Gruppen) ---")
        print(calibration)
        print("\n(Idealfall: 'Ø Predicted Prob' ist fast gleich 'Ø Observed Mortality')")
    except Exception as e:
        print(f"Konnte Kalibrierungstabelle nicht erstellen (evtl. zu wenige Daten/Varianz): {e}")

    # D) AUROC (Discrimination) - Wie gut trennt der Score?
    try:
        from sklearn.metrics import roc_auc_score
        # Filtere NaNs raus für die Berechnung
        mask = df_aki['sapsii'].notnull() & df_aki['hospital_mortality'].notnull()
        auroc = roc_auc_score(df_aki.loc[mask, 'hospital_mortality'], df_aki.loc[mask, 'sapsii'])
        print(f"\n--- Trennschärfe ---")
        print(f"AUROC Score: {auroc:.3f}")
        print("(0.5 = Zufall, >0.7 = Gut, >0.8 = Sehr gut)")
    except ImportError:
        print("\nscikit-learn nicht installiert, überspringe AUROC.")


Rows: 10485

SAPS II PLAUSIBILITÄTS-CHECK

--- Durchschnittlicher SAPS II Score ---
                         mean  median        std  count
hospital_mortality                                     
0                   39.514652    39.0  13.039172   8190
1                   54.149891    53.0  16.134440   2295

--- Global Metrics ---
Beobachtete Mortalität:  21.89%
Vorhergesagte Mortalität: 33.40%
SMR (Observed/Predicted): 0.655
-> SMR ≈ 1.0: Score passt perfekt.
-> SMR < 1.0: Weniger Patienten gestorben als vorhergesagt (gute Performance/Score überschätzt Risiko).
-> SMR > 1.0: Mehr Patienten gestorben als vorhergesagt (Score unterschätzt Risiko).
Konnte Kalibrierungstabelle nicht erstellen (evtl. zu wenige Daten/Varianz): "Columns not found: 'hospital_expire_flag'"

--- Trennschärfe ---
AUROC Score: 0.762
(0.5 = Zufall, >0.7 = Gut, >0.8 = Sehr gut)
